In [1]:
import numpy as np
import pathlib
import pandas as pd

import tensorflow as tf
import optuna
from sklearn.preprocessing import MinMaxScaler, RobustScaler, Normalizer

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
N_INPUTS = 19
N_CLASSES = 3

In [4]:
def objective(trial):
    cleaned_data_path = pathlib.Path("../data/interim/ctg_cleaned_manually.xlsx")
    data = pd.read_excel(cleaned_data_path,header=0)
    x_raw = data.loc[:, "Min":"SUSP"].to_numpy()
    y_raw_nsp = data.loc[:, "NSP"].to_numpy()
    y_nsp_one_hot = tf.one_hot(indices=y_raw_nsp,depth=max(y_raw_nsp)).numpy()

    x_train = x_raw[:1913]
    x_test = x_raw[1913:]
    y_train = y_nsp_one_hot[:1913]
    y_test = y_nsp_one_hot[1913:]
    
    n_layers = trial.suggest_int('n_layers', 1, 3)
    batch_size = trial.suggest_int('batch_size', 1, 64,log=True)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    epochs = trial.suggest_int("epochs", 5,50,5)
    batch_norm = trial.suggest_categorical("batch norm",[True, False])
    normalize = trial.suggest_categorical("normalize",["minmax","meanstd","robust"])
    
    if normalize == "minmax":
        scaler = MinMaxScaler()
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
    elif normalize == "meanstd":
        scaler = Normalizer()
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
    elif normalize == "robust":
        scaler = RobustScaler()
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=N_INPUTS,activation="relu"))
    
    for i in range(n_layers):
        num_hidden = trial.suggest_int('n_units_l{}'.format(i), 4, 128,log=True)
        model.add(tf.keras.layers.Dense(num_hidden, activation='relu'))
        if batch_norm:
            model.add(tf.keras.layers.BatchNormalization())
        
    model.add(tf.keras.layers.Dense(N_CLASSES,activation="sigmoid"))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss="categorical_crossentropy",
                  metrics="categorical_accuracy")
    model.fit(x=x_train,
          y=y_train,
          batch_size=batch_size,
          validation_split=0.2,
          epochs=epochs,
         verbose=0)
    [loss, accuracy] = model.evaluate(x=x_test,y=y_test)
    return accuracy
    

In [7]:
study = optuna.create_study(storage="sqlite:///../database/study/keras_mlp.sqlite",
                            sampler=optuna.samplers.TPESampler(seed=42),
                            direction="maximize", 
                            study_name="keras_mlp",
                            load_if_exists=True)
study.optimize(objective, n_trials=1000,n_jobs=1)

[I 2021-07-19 13:56:20,582] Using an existing study with name 'keras_mlp' instead of creating a new one.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

7/7 [==============================] - 0s 1ms/step - loss: 0.0891 - categorical_accuracy: 0.8028


[I 2021-07-19 13:56:42,868] Trial 44 finished with value: 0.8028169274330139 and parameters: {'n_layers': 3, 'batch_size': 13, 'learning_rate': 0.0006716343877780903, 'epochs': 20, 'batch norm': True, 'normalize': 'minmax', 'n_units_l0': 14, 'n_units_l1': 51, 'n_units_l2': 62}. Best is trial 43 with value: 0.9953051805496216.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

7/7 [==============================] - 0s 1ms/step - loss: 0.4740 - categorical_accuracy: 0.6244


[I 2021-07-19 13:56:59,070] Trial 45 finished with value: 0.6244131326675415 and parameters: {'n_layers': 3, 'batch_size': 23, 'learning_rate': 9.168674565516075e-05, 'epochs': 15, 'batch norm': True, 'normalize': 'minmax', 'n_units_l0': 27, 'n_units_l1': 77, 'n_units_l2': 16}. Best is trial 43 with value: 0.9953051805496216.


...


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

7/7 [==============================] - 0s 1ms/step - loss: 2.7822 - categorical_accuracy: 0.2488


[I 2021-07-19 14:13:08,384] Trial 85 finished with value: 0.2488262951374054 and parameters: {'n_layers': 3, 'batch_size': 28, 'learning_rate': 0.00035832982486850386, 'epochs': 20, 'batch norm': True, 'normalize': 'meanstd', 'n_units_l0': 9, 'n_units_l1': 63, 'n_units_l2': 16}. Best is trial 43 with value: 0.9953051805496216.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

7/7 [==============================] - 0s 1ms/step - loss: 13.1734 - categorical_accuracy: 0.1737


[I 2021-07-19 14:13:32,697] Trial 86 finished with value: 0.17370891571044922 and parameters: {'n_layers': 3, 'batch_size': 1, 'learning_rate': 0.0006684066210971711, 'epochs': 5, 'batch norm': True, 'normalize': 'minmax', 'n_units_l0': 19, 'n_units_l1': 88, 'n_units_l2': 70}. Best is trial 43 with value: 0.9953051805496216.


KeyboardInterrupt: 